<a href="https://colab.research.google.com/github/asishpattnaik1/CE888/blob/main/Lab_8/TF_Feature_Extractor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget https://raw.githubusercontent.com/sagihaider/CE888_2021/main/Lab_8/data.zip

In [ ]:
!unzip data.zip

In [ ]:
import os
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [ ]:
print(len(os.listdir('./data/Humans/')))
print(len(os.listdir('./data/cats/')))
print(len(os.listdir('./data/dogs/')))
print(len(os.listdir('./data/horses/')))

In [ ]:
ath_humans = './data/Humans'
path_cats = './data/cats'
path_dogs = './data/dogs/'
path_horses = './data/horses/'

humans = os.listdir(path_humans)
cats = os.listdir(path_cats)
dogs = os.listdir(path_dogs)
horses = os.listdir(path_horses)

human_image_sizes = []
cats_image_sizes = []
dogs_image_sizes = []
horses_image_sizes = []

for i in range(202):
  hu = plt.imread(os.path.join(path_humans, humans[i]))
  c = plt.imread(os.path.join(path_cats, cats[i]))
  d = plt.imread(os.path.join(path_dogs, dogs[i]))
  ho = plt.imread(os.path.join(path_horses, horses[i]))
  human_image_sizes.append(hu.shape[:-1])
  cats_image_sizes.append(c.shape[:-1])
  dogs_image_sizes.append(d.shape[:-1])
  horses_image_sizes.append(ho.shape[:-1])

In [ ]:
mean_sizes = [np.mean(human_image_sizes, axis=0), np.mean(cats_image_sizes, axis=0),
              np.mean(dogs_image_sizes, axis=0), np.mean(horses_image_sizes, axis=0)]

target_size = np.array(mean_sizes).mean(axis=0)
target_size

In [ ]:
image_generator = ImageDataGenerator(rescale=1.0/255, 
                               rotation_range=20,
                               horizontal_flip=True, 
                               validation_split=0.1)

In [ ]:
train_data = image_generator.flow_from_directory('./data/', target_size=(304,329),
                                      batch_size=4,subset='training')

val_data = image_generator.flow_from_directory('./data/', target_size=(304,329),
                                      batch_size=4,subset='validation')

In [ ]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, Input, Flatten
from tensorflow.keras import Model

image_size = [304,329,3]

In [ ]:
input = Input(shape = image_size)
model = ResNet50(input_tensor=input,)

In [ ]:
odel.summary()

In [ ]:
last_layer = model.get_layer('avg_pool').output
out = Dense(4, activation='softmax', name='output')(last_layer)
model = Model(input, out)

for layer in model.layers[:-1]:
  layer.trainable = False


model.summary()

In [ ]:

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics="accuracy")

In [ ]:
my_callbacks = [
    EarlyStopping(monitor="val_accuracy", patience=10, restore_best_weights=True),
    ModelCheckpoint(filepath='vgg16_model.h5', save_best_only=True),
]

In [ ]:
model.fit(train_data, validation_data=val_data, epochs=50, callbacks=my_callbacks)

In [ ]:
model.history.history